In [5]:
# import module

import os
import re
from tqdm import tqdm

import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
# from skimage.measure import compare_ssim
from skimage.metrics import structural_similarity as ssim
import argparse
import imutils

from tqdm import tqdm

In [9]:
# 파일 이름 불러오기
img_lst = []
lst_age = [20,30,40,50]
lst_sex = ['man','woman']
for age in lst_age:
    for sex in lst_sex:
        img_lst.append(glob(f'image/*.jpg'))
        
img_lst_final = glob('images_final/*.jpg')

In [12]:
# SSIM Test
imageA = cv2.imread(img_lst[0][0])
imageB = cv2.imread(img_lst_final[1])
imageA = cv2.resize(imageA, (128,128))
imageB = cv2.resize(imageB, (128,128))
grayA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
grayB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
(score, diff) = ssim(grayA, grayB, full=True)
diff = (diff * 255).astype("uint8")

print("SSIM : {}".format(score))

SSIM : 0.5127219714707076


In [14]:
# 빈 리스트 생성
lst_ssim = []

# 한 이미지에 대한 SSIM 점수 측정.
for lst in tqdm(img_lst):
    for i in lst:
        for img in img_lst_final:
            imageA = cv2.imread(lst[0])
            imageB = cv2.imread(img)
            imageC = cv2.resize(imageA, (128,128))
            imageD = cv2.resize(imageB, (128,128))
            grayC = cv2.cvtColor(imageC, cv2.COLOR_BGR2GRAY)
            grayD = cv2.cvtColor(imageD, cv2.COLOR_BGR2GRAY)
            (score, diff) = ssim(grayC, grayD, full=True)
            diff = (diff * 255).astype("uint8")
            lst_ssim.append([i[13:],img[13:], score, sex, age])

100%|████████████████████████████████████████| 8/8 [39:31<00:00, 296.45s/it]


In [18]:
# 한 이미지에 대한 SSIM 값 Top 5.
df_ssim = pd.DataFrame(lst_ssim, columns=['basic_images','target_images','score','sex','age'])
df_ssim = df_ssim.sort_values(by=['score'], axis=0,ascending=False)[1:6]; df_ssim

,basic_images,target_images,score,sex,age
199525,00158.jpg,20201030-img_00000028 8_0.jpg,0.595981,woman,50
217800,09854.jpg,20201030-img_00000028 8_0.jpg,0.595981,woman,50
46015,08716.jpg,20201030-img_00000028 8_0.jpg,0.595981,woman,50
122770,09752.jpg,20201030-img_00000028 8_0.jpg,0.595981,woman,50
108150,08958.jpg,20201030-img_00000028 8_0.jpg,0.595981,woman,50


In [19]:
# 가중평균(weighted average)
weight = []
for score in df_ssim['score']:
    weight.append(score/df_ssim['score'].sum())
    
# 각 weighted 값 계산
weight

[0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998,
 0.19999999999999998]

In [20]:
# dim 값 불러오기
img_factor = pd.read_csv("tables/img_factors.csv")
item_factor = pd.read_csv("tables/item_factors.csv")

In [22]:
# 각 이미지에 대한 임베딩 값
img_top5_dim = []
for dim in df_ssim['target_images']:
    img_top5_dim.append(img_factor.query(f'target == "{dim}"').iloc[0])
df_img_ssim = pd.DataFrame(img_top5_dim);df_img_ssim

,target,crop,dim01,dim02,dim03,dim04,dim05,dim06,dim07,dim08,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
2293,20201030-img_00000028 8_0.jpg,20201030-img_00000028 8_3.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,20201030-img_00000028 8_0.jpg,20201030-img_00000028 8_3.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,20201030-img_00000028 8_0.jpg,20201030-img_00000028 8_3.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,20201030-img_00000028 8_0.jpg,20201030-img_00000028 8_3.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,20201030-img_00000028 8_0.jpg,20201030-img_00000028 8_3.jpg,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365


In [23]:
# 임베딩값만 가져오기
img_emb_value = df_img_ssim.iloc[:,2:]

In [24]:
img_emb_value

,dim01,dim02,dim03,dim04,dim05,dim06,dim07,dim08,dim09,dim10,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365


In [25]:
# 산술평균(arithmetic mean)
img_arithmetic_mean = []
for dim in img_emb_value.columns:
    img_arithmetic_mean.append(img_emb_value[dim].sum()/5)

In [26]:
# 임베딩 값 추가
# img_imb_value.append(pd.DataFrame([img_arithmetic_mean], index=['arithmetic_mean'], columns=img_imb_value.columns))

In [27]:
# 한 이미지에 대한 가중평균 값.
img_w_avg = []
for dim,w in zip(range(len(img_emb_value.index)), weight):
    img_w_avg.append(img_emb_value.loc[img_emb_value.index[dim]]*w)
df_img_w_avg = sum(img_w_avg);df_img_w_avg

,dim01,dim02,dim03,dim04,dim05,dim06,dim07,dim08,dim09,dim10,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365
2293,0.31746,0.31746,0.563492,0.849206,0.468254,0.079365,0.68254,0.984127,0.111111,0.793651,...,0.515873,0.190476,0.47619,0.84127,0.333333,1.0,0.0,0.230159,0.865079,0.079365


In [29]:
item_top5_dim = []
for dim in df_ssim['target_images']:
    item_top5_dim.append(item_factor.query(f'target == "{dim}"').iloc[0])
df_ssim_item = pd.DataFrame(item_top5_dim);df_ssim_item

,target,dim01,dim02,dim03,dim04,dim05,dim06,dim07,dim08,dim09,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
2829,20201030-img_00000028 8_0.jpg,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
2829,20201030-img_00000028 8_0.jpg,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
2829,20201030-img_00000028 8_0.jpg,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
2829,20201030-img_00000028 8_0.jpg,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
2829,20201030-img_00000028 8_0.jpg,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144


In [30]:
# 임베딩값만 가져오기
emb_value_item = df_ssim_item.iloc[:,1:]

# user_factor 산술평균(arithmetic mean)
arithmetic_mean_item = []
for dim in emb_value_item.columns:
    arithmetic_mean_item.append(emb_value_item[dim].sum()/5)

In [31]:
# 임베딩 값 추가
# imb_value_item.append(pd.DataFrame([arithmetic_mean_item], index=['arithmetic_mean'], columns=imb_value_item.columns))

In [32]:
# 한 이미지에 대한 가중평균 값.
df_item_w_avg = []
for dim, w in zip(range(len(emb_value_item.index)), weight):
    df_item_w_avg.append(emb_value_item.loc[emb_value_item.index[dim]]*w)
df_item_w_average = sum(df_item_w_avg);df_item_w_average

,dim01,dim02,dim03,dim04,dim05,dim06,dim07,dim08,dim09,dim10,...,dim23,dim24,dim25,dim26,dim27,dim28,dim29,dim30,dim31,dim32
2829,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,0.22747,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
2829,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,0.22747,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
2829,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,0.22747,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
2829,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,0.22747,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
2829,-0.199382,0.270453,0.251533,0.338618,-0.336595,-0.224803,-0.074305,-0.035659,-0.287146,0.22747,...,-0.287342,0.098859,-0.320112,0.118643,-0.077065,-0.282613,0.097118,-0.153842,0.261499,0.146144
